In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings                                  
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
import matplotlib.pyplot as plot
import matplotlib.colors
from os.path import join
import pprint
pp = pprint.PrettyPrinter(indent=4)

from src.data.data import *
data = data_wrapper()

deliveries_path = join(data.interimDirectory, 'deliveries', 'deliveries.yaml')
with open(deliveries_path, 'r') as deliveries_yaml:
    print (f'Loading deliveries file from: {deliveries_path}')
    deliveries = yaml.load(deliveries_yaml, Loader=yaml.SafeLoader)

In [ ]:
# INPUT DATA
# Name of test to be analysed
client = 'ILLA_DE_AROUSA'
batch = 1

type_file = None
# Percentage of points to be considered NG sensor
limit_errors = 3
# Multiplier for std_dev (sigma) - Normal distribution (99.73%)
limit_confidence_sigma = 3
# t-student confidence level (%)
t_confidence_level = 99
# Use average dispersion or instantaneous
use_instantatenous_dispersion = False
# Min/max date for the analysis
# min_date = '2019-12-10 18:00:00'
min_date = None
max_date = None
# In case there is a device with lower amount of channels, ignore the missing channels and keep going
ignore_missing_channels = True
# Smooth channels
smooth_channels = True
smooth_number = 5

In [ ]:
display(HTML('<p><strong>Delivery:</strong><br> {}</p>'.format(client)))
display(HTML('<p><strong>Delivery:</strong><br> {}</p>'.format(client)))
display(HTML('<p><strong>Total number of devices tested:</strong><br> {}</p>'.format(len(list_devices))))
display(HTML('<p><strong>Test dates:</strong><br> {} - {}</p>'.format(min_date, max_date)))
display(HTML('<p><strong>Test author(s):</strong><br> {}</p><hr>'.format('Óscar González - Victor Barberán')))

### Test Explanation

The devices are co-located for a period of at least 3 days in an **indoor** environment. Devices that show an abnormal behaviour are analysed and replaced if necessary. **In this case, the devices were tested for 2+ weeks during Christmas holidays**.

#### Comment on methods
- This analysis is conditioned to the low amount of devices tested. The plots below indicate only trends and highlight some (but not all) potential faulty devices.
- Below, the confidence intervals used are those of the normal distribution (sample numbers >30) and of the t-student distribution (sample numbers <30).
The testing of the devices is done for at least 3-5 days. If any problem is seen, longer periods are tested if needed. Also note that statistical significance cannot be inferred from these amounts of devices, and some assumptions have to be accepted. Comments and further discussion are always welcome.
- The individual sensors components integrated in the Smart Citizen hardware have their own accuracies and dispersions, for which Smart Citizen cannot assume any liability other than trying to work with the most appropiate selection. The tests we perform are aimed to determine and assume any failures in the sensors and their integration within the Smart Citizen hardware. For more information, please check the <a href="https://docs.smartcitizen.me">official documentation</a> and the datasheets of each of the sensors in the [sensors part](https://docs.smartcitizen.me/Components/Urban%20Sensor%20Board/).